Homework

In [1]:
#1. Read the data for January. How many columns are there?

import pandas as pd
df_jan=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_feb=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
len(df_jan.columns)

19

In [2]:
#2. Now let's compute the duration variable. It should contain the duration of a ride in minutes.
#What's the standard deviation of the trips duration in January?

df_jan['duration']=df_jan.tpep_dropoff_datetime-df_jan.tpep_pickup_datetime
df_jan['duration'] = df_jan.duration.dt.total_seconds() / 60
df_jan.duration.std()

42.59435124195458

In [3]:
#3. we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records 
# where the duration was between 1 and 60 minutes (inclusive).
# What fraction of the records left after you dropped the outliers?
total_records_before = len(df_jan)
total_records_before

3066766

In [4]:
df_jan_filtered = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)].copy()
total_records_after = len(df_jan_filtered)
total_records_after

3009173

In [5]:
fraction_left = total_records_after / total_records_before
print(f"Fraction of records left after dropping outliers: {fraction_left:.2f}")

Fraction of records left after dropping outliers: 0.98


In [6]:
#4.
#Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
#Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
#Fit a dictionary vectorizer
#Get a feature matrix from it

from sklearn.feature_extraction import DictVectorizer

df_jan
categorical = ['PULocationID','DOLocationID']

#change to object
df_jan[categorical] = df_jan[categorical].astype(str)

#change to dictionary
train_dicts = df_jan[categorical].to_dict(orient='records')


dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape


(3066766, 518)

In [7]:
#5.Now let's use the feature matrix from the previous step to train a model.

#Train a plain linear regression model with default parameters, where duration is the response variable
#Calculate the RMSE of the model on the training data
#What's the RMSE on train?

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

target='duration'
y_train=df_jan[target].values

lr=LinearRegression()
lr.fit(X_train,y_train)

y_pred=lr.predict(X_train)
mean_squared_error(y_train,y_pred,squared=False)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


41.99649772007039

In [8]:
#6. Now let's apply this model to the validation dataset (February 2023).
#What's the RMSE on validation?


categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    df[categorical] = df[categorical].astype('str')
    return df

df_val = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts) 
y_val = df_val.duration.values
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.94521018617827